# 🕌 Hadith Question-Answering System

**بِسْمِ ٱللهِ ٱلرَّحْمٰنِ ٱلرَّحِيْمِ**

This notebook implements an intelligent Hadith search and question-answering system using:
- **Semantic Search**: Sentence Transformers for embeddings
- **Vector Database**: FAISS for fast similarity search
- **AI Responses**: Groq API with LLM for contextual answers
- **Authentic Data**: 39,038 Hadiths from six canonical collections

---

## 📚 Data Sources
- Sahih Bukhari
- Sahih Muslim  
- Abu Daud
- Ibn Majah
- Al-Nasa'i
- Jami' at-Tirmidhi

In [15]:
import pandas as pd
import glob
import re
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
import os

## 1️⃣ Import Required Libraries

Import all necessary libraries for data processing, embeddings, and vector search.

In [16]:
csv_path = glob.glob("data/**/*.csv")

## 2️⃣ Load Hadith Data from CSV Files

Collect all CSV files from the data directory containing Hadith collections.

In [17]:
df_list = [pd.read_csv(f) for f in csv_path]
df = pd.concat(df_list,ignore_index=True)


## 3️⃣ Concatenate All Data into Single DataFrame

Combine all Hadith collections into one unified dataset for processing.

In [18]:
df.dropna(subset=['English_Hadith', 'Arabic_Hadith'], inplace=True)
df = df[df['English_Hadith'].str.strip() != '']
df = df[df['Arabic_Hadith'].str.strip() != '']

## 4️⃣ Clean Data

Remove null values and empty strings from English and Arabic Hadith columns.

In [19]:
def clean_text(text):
    if isinstance(text, str):
        text = re.sub(r'[^A-Za-z\s]', '', text)
        text = text.lower()
    else:
        text = ''
    return text

## 5️⃣ Define Text Cleaning Function

Normalize text by removing special characters and converting to lowercase for consistent embeddings.

In [20]:
df['English_Hadith'] = df['English_Hadith'].apply(clean_text)

## 6️⃣ Apply Text Cleaning

Clean all English Hadith texts to prepare for embedding generation.

In [ ]:
english_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
arabic_model = SentenceTransformer('aubmindlab/bert-base-arabertv2')

## 7️⃣ Load Sentence Transformer Models

Initialize pre-trained models:
- **English**: MiniLM-L6-v2 (384-dimensional embeddings)
- **Arabic**: AraBERT v2 (768-dimensional embeddings)

In [22]:
df.head()

,Chapter_Number,Chapter_English,Chapter_Arabic,Section_Number,Section_English,Section_Arabic,Hadith_number,English_Hadith,English_Isnad,English_Matn,Arabic_Hadith,Arabic_Isnad,Arabic_Matn,Arabic_Comment,English_Grade,Arabic_Grade
0,27.0,Chapters on Slaughtering,كتاب الذبائح,NaN,NaN,NaN,3207,it was narrated that umm kurz said i heard the...,It was narrated that Umm Kurz said:,“I heard the Prophet (ﷺ) say: ‘On behalf of a ...,حَدَّثَنَا أَبُو بَكْرِ بْنُ أَبِي شَيْبَةَ، و...,حَدَّثَنَا أَبُو بَكْرِ بْنُ أَبِي شَيْبَةَ، و...,أُمِّ كُرْزٍ، قَالَتْ سَمِعْتُ النَّبِيَّ صلى ...,,Hasan - Good,حسن
1,27.0,Chapters on Slaughtering,كتاب الذبائح,NaN,NaN,NaN,3208,it was narrated that aishah said the messenger...,It was narrated that ‘Aishah said:,“The Messenger of Allah (ﷺ) commanded us to sa...,حَدَّثَنَا أَبُو بَكْرِ بْنُ أَبِي شَيْبَةَ، ح...,حَدَّثَنَا أَبُو بَكْرِ بْنُ أَبِي شَيْبَةَ، ح...,عَائِشَةَ، قَالَتْ أَمَرَنَا رَسُولُ اللَّهِ ص...,,Hasan - Good,حسن
2,27.0,Chapters on Slaughtering,كتاب الذبائح,NaN,NaN,NaN,3209,it was narrated from salman bin amir that he h...,It was narrated from Salman bin ‘Amir that he ...,"“For a boy there should be an ‘Aqiqah, so shed...",حَدَّثَنَا أَبُو بَكْرِ بْنُ أَبِي شَيْبَةَ، ح...,حَدَّثَنَا أَبُو بَكْرِ بْنُ أَبِي شَيْبَةَ، ح...,سَمِعَ النَّبِيَّ صلى الله عليه وسلم يَقُولُ إ...,,Sahih - Authentic,صحيح
3,27.0,Chapters on Slaughtering,كتاب الذبائح,NaN,NaN,NaN,3210,it was narrated from samurah that the prophet ...,It was narrated from Samurah that the Prophet ...,"“Every boy is mortgaged by his ‘Aqiqah, so sla...",حَدَّثَنَا هِشَامُ بْنُ عَمَّارٍ، حَدَّثَنَا ش...,حَدَّثَنَا هِشَامُ بْنُ عَمَّارٍ، حَدَّثَنَا ش...,عَنِ النَّبِيِّ صلى الله عليه وسلم قَالَ كُلُّ...,,Hasan - Good,حسن
4,27.0,Chapters on Slaughtering,كتاب الذبائح,NaN,NaN,NaN,3211,yazid bin abdulmuzani narrated that the prophe...,Yazid bin ‘Abdul-Muzani narrated that the Prop...,"“Offer an ‘Aqiqah for the boy, but do not smea...",حَدَّثَنَا يَعْقُوبُ بْنُ حُمَيْدِ بْنِ كَاسِب...,حَدَّثَنَا يَعْقُوبُ بْنُ حُمَيْدِ بْنِ كَاسِب...,أَنَّ النَّبِيَّ صلى الله عليه وسلم قَالَ يُعَ...,,Hasan - Good,حسن


## 8️⃣ Preview Dataset

Display the first few rows to verify data structure and content.

In [ ]:
if os.path.exists('english_embeddings.npy') and os.path.exists('arabic_embeddings.npy'):
    df['English_Embedding'] = np.load('english_embeddings.npy').tolist()
    df['Arabic_Embedding'] = np.load('arabic_embeddings.npy').tolist()
else:
    df['English_Embedding'] = df['English_Hadith'].apply(lambda x: english_model.encode(x, convert_to_numpy=True))
    df['Arabic_Embedding'] = df['Arabic_Hadith'].apply(lambda x: arabic_model.encode(x, convert_to_numpy=True))
    np.save('english_embeddings.npy', np.array(df['English_Embedding'].tolist()))
    np.save('arabic_embeddings.npy', np.array(df['Arabic_Embedding'].tolist()))

## 9️⃣ Generate or Load Embeddings

Create vector embeddings for all Hadiths (or load pre-computed ones):
- Enables semantic similarity search
- Cached to disk for faster subsequent runs

In [ ]:
embeddings_en = np.load('english_embeddings.npy')
d_en = embeddings_en.shape[1]
index_en = faiss.IndexFlatL2(d_en)
index_en.add(embeddings_en)
faiss.write_index(index_en, 'hadith_faiss_en.index')


embeddings_ar = np.load('arabic_embeddings.npy')
d_ar = embeddings_ar.shape[1]
index_ar = faiss.IndexFlatL2(d_ar)
index_ar.add(embeddings_ar)
faiss.write_index(index_ar, 'hadith_faiss_ar.index')

## 🔟 Build FAISS Vector Indices

Create FAISS indices for ultra-fast similarity search:
- **L2 Distance**: Measures Euclidean distance between vectors
- Separate indices for English and Arabic

In [ ]:
features = ["Chapter_Number","Hadith_number","English_Hadith","Arabic_Hadith","English_Isnad","English_Matn","English_Grade"]

def retrieve_similar_hadiths(query, model, index, hadith_df, k=5):
    query_embedding = model.encode([clean_text(query)])
    distances, indices = index.search(query_embedding, k)
    for i in range(k):
        print(f"Hadith {i+1}:")
        print("English:", hadith_df['English_Hadith'].iloc[indices[0][i]])
        print("Arabic:", hadith_df['Arabic_Hadith'].iloc[indices[0][i]])
        print("-----------------------")
        for col in features:
            print(f'{col} : {hadith_df[col].iloc[indices[0][i]]}')
        print(f"Distance: {distances[0][i]}\n")
    

## 1️⃣1️⃣ Define Hadith Retrieval Function

Function to retrieve and display similar Hadiths based on query:
- Encodes query using sentence transformer
- Searches FAISS index for nearest neighbors
- Returns top K most similar Hadiths

In [ ]:
query = "Fasting in Ramadan"
retrieve_similar_hadiths(query, english_model, index_en, df)


Hadith 1:
English: narrated abu ayyub the prophet  as saying if anyone fasts during ramadan then follows it with six days in shawwal it will be like a perpetual fast
Arabic: حَدَّثَنَا النُّفَيْلِيُّ، حَدَّثَنَا عَبْدُ الْعَزِيزِ بْنُ مُحَمَّدٍ، عَنْ صَفْوَانَ بْنِ سُلَيْمٍ، وَسَعْدِ بْنِ سَعِيدٍ، عَنْ عُمَرَ بْنِ ثَابِتٍ الأَنْصَارِيِّ، عَنْ أَبِي أَيُّوبَ، صَاحِبِ النَّبِيِّ صلى الله عليه وسلم عَنِ النَّبِيِّ صلى الله عليه وسلم قَالَ ‏ ‏ مَنْ صَامَ رَمَضَانَ ثُمَّ أَتْبَعَهُ بِسِتٍّ مِنْ شَوَّالٍ فَكَأَنَّمَا صَامَ الدَّهْرَ ‏‏ ‏.‏
-----------------------
Chapter_Number : 14.0
Hadith_number : 2299
English_Hadith : narrated abu ayyub the prophet  as saying if anyone fasts during ramadan then follows it with six days in shawwal it will be like a perpetual fast
Arabic_Hadith : حَدَّثَنَا النُّفَيْلِيُّ، حَدَّثَنَا عَبْدُ الْعَزِيزِ بْنُ مُحَمَّدٍ، عَنْ صَفْوَانَ بْنِ سُلَيْمٍ، وَسَعْدِ بْنِ سَعِيدٍ، عَنْ عُمَرَ بْنِ ثَابِتٍ الأَنْصَارِيِّ، عَنْ أَبِي أَيُّوبَ، صَاحِبِ النَّبِيِّ صلى الل

## 1️⃣2️⃣ Test Retrieval System

Example search to verify the system is working correctly.

In [ ]:
from dotenv import load_dotenv
import os
from groq import Groq

# Load .env
load_dotenv()

# Initialize Groq client
client = Groq(api_key=os.getenv("GROQ_API_KEY"))

# Define system prompts
concise_prompt = "You are a respectful assistant who answers only using authentic Hadith from Sahih Bukhari, Sahih Muslim, and other authentic collections (such as Abu Daud, Ibn Majah, Al-Nasa'i, and Jami' at-Tirmidhi). Your responses must be strictly based on the provided context. Do not add external knowledge, assumptions, or information outside this context. The context contains Hadiths, each with the Hadith text, Isnad, and Grade separated by newlines. Keep answers concise, presenting the Hadiths in a numbered list. Format each Hadith as follows:\n\n1. [Hadith text]\n\n   Narrated by: [Isnad]\n\n   Grade: [Grade]\n\nSummarize key points without elaboration."

detailed_prompt = "You are a respectful assistant who answers only using authentic Hadith from Sahih Bukhari, Sahih Muslim, and other authentic collections (such as Abu Daud, Ibn Majah, Al-Nasa'i, and Jami' at-Tirmidhi). Your responses must be strictly based on the provided context. Do not add external knowledge, assumptions, or information outside this context. Provide detailed answers, quoting all relevant parts of the context and explaining their relevance step-by-step."

# Select choice (change this to "detailed" if needed)
choice = "detailed"

# Set system content and model based on choice
if choice == "concise":
    system_content = concise_prompt
    model_name = "meta-llama/llama-4-scout-17b-16e-instruct"
    top_k = 3  # Top 3 for concise
elif choice == "detailed":
    system_content = detailed_prompt
    model_name = "openai/gpt-oss-120b"
    top_k = 5  # All 5 for detailed
else:
    system_content = concise_prompt  # Default to concise
    model_name = "meta-llama/llama-4-scout-17b-16e-instruct"
    top_k = 3

# Function to retrieve similar hadiths and return data
def retrieve_similar_hadiths_data(query, model, index, hadith_df, k=5):
    query_embedding = model.encode([clean_text(query)])
    distances, indices = index.search(query_embedding, k)
    results = []
    for i in range(k):
        hadith_data = {
            "Chapter_Number": hadith_df['Chapter_Number'].iloc[indices[0][i]],
            "Hadith_number": hadith_df['Hadith_number'].iloc[indices[0][i]],
            "English_Hadith": hadith_df['English_Hadith'].iloc[indices[0][i]],
            "Arabic_Hadith": hadith_df['Arabic_Hadith'].iloc[indices[0][i]],
            "English_Isnad": hadith_df['English_Isnad'].iloc[indices[0][i]],
            "English_Matn": hadith_df['English_Matn'].iloc[indices[0][i]],
            "English_Grade": hadith_df['English_Grade'].iloc[indices[0][i]],
            "Distance": distances[0][i]
        }
        results.append(hadith_data)
    return results

# Chatbot loop
print("Welcome to the Hadith Chatbot! Ask questions about Hadith. Type 'quit' to exit.")
while True:
    query = input("You: ")
    if query.lower() == 'quit':
        print("Chatbot: Goodbye!")
        break

    # Retrieve top 5 hadiths
    retrieved_hadiths = retrieve_similar_hadiths_data(query, english_model, index_en, df, k=5)

    # Select top_k based on choice
    selected_hadiths = retrieved_hadiths[:top_k]

    # Format context
    context = "\n\n".join([
        f"{h['English_Hadith']}\n{h['English_Isnad']}\n{h['English_Grade']}"
        for h in selected_hadiths
    ])

    messages = [
        {"role": "system", "content": system_content},
        {"role": "user", "content": f"Context:\n{context}\n\nQuery: {query}"},
    ]

    response = client.chat.completions.create(
        model=model_name,
        messages=messages
    )

    print(f"Chatbot: {response.choices[0].message.content}")

Welcome to the Hadith Chatbot! Ask questions about Hadith. Type 'quit' to exit.


Chatbot: **What the authentic ḥadīth say about fasting in Ramadan (and just after it)**  

Below are the ḥadīth that were supplied in the context, each of them is marked **Sahih – Authentic**. From these narrations we can extract the rulings and the special reward that the Prophet  ﷺ described.

---

### 1.  When the fast of Ramadan must begin and end  

**Narrated Abu Hurairah**  
> “It was narrated from Abu Hurairah that the Messenger of Allah (ﷺ) said: **‘Do not anticipate Ramadan by fasting one or two days before, except for a man who has a habitual pattern of fasting; in that case let him fast.’**”  
> *Sahih – Authentic*

**Narrated Ibn ‘Abbas**  
> “It was narrated from Ibn ‘Abbas that the Messenger of Allah (ﷺ) said: **‘Do not fast before Ramadan; fast with its sighting and break fast with its sighting, and if it is obscured from you then complete thirty days.’**”  
> *Sahih – Authentic*

**Explanation step‑by‑step**

1. **No pre‑Ramadan voluntary fasts** – The Prophet (ﷺ) forb

## 1️⃣3️⃣ AI-Powered Hadith Chatbot

Interactive chatbot using:
- **Retrieval**: FAISS vector search
- **LLM**: Groq API for intelligent responses
- **Two Modes**:
  - **Concise**: Brief answers with top 3 Hadiths
  - **Detailed**: Comprehensive explanations with top 5 Hadiths

### System Prompts
Configured to provide contextual, authentic responses based only on retrieved Hadiths.